In [5]:
import pandas as pd
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [6]:
portal_client = get_portal_client()

In [7]:
existing_missing_df = pd.read_csv("data/2025-08-04 Masterfire Missing Tasks New.csv").sort_values("site_id")

In [8]:
missing_sites = existing_missing_df.loc[existing_missing_df["missing"] == 1]

In [5]:
problem_sites = []

for site_id in missing_sites["site_id"]:
    print(f"Site {site_id}")
    if site_id in [8466]:
        print("Skipping known problematic site")
        continue
    problem_tests = portal_client.query_single_value(
        """SELECT
            COUNT(DISTINCT test.id)
        FROM topaz.api_service_tests AS test
        INNER JOIN topaz.api_mode_changes AS mode_change
        ON test.mode_change_id = mode_change.id
        INNER JOIN topaz.api_sites AS site
        ON test.site_id = site.id
        INNER JOIN topaz.api_toc_rows AS toc
        ON (mode_change.toc_id = toc.toc_id)
        AND (
            (test.domain = toc.domain) OR
            (toc.domain IS NULL)
        )
        AND (
            (test.panel = toc.panel) OR
            (toc.panel IS NULL)
        )
        AND (
            (test.loop = toc.loop) OR
            (toc.loop IS NULL)
        )
        AND (test.address = toc.address)
        AND (test.address_type = toc.address_type)
        AND (
            (
                (
                    (test.sub_address = '') OR
                    (test.sub_address IS NULL)
                ) AND (
                    (toc.sub_address IS NULL) OR
                    (toc.sub_address = 0)
                )
            ) OR
            (test.sub_address = toc.sub_address)
        )
        AND test.created_at BETWEEN toc.created_at AND IFNULL(toc.deleted_at, CURRENT_TIMESTAMP())
        INNER JOIN topaz.api_service_tasks AS task_template
        ON task_template.template_id = site.template_id
        AND (
            (
                (task_template.asset_type = toc.lcs_device_type)
                AND (toc.lcs_device_type!= 'Other')
            )
            OR (
                toc.lcs_device_type = 'Other'
                AND (toc.type = task_template.asset_type)
            )
            OR (task_template.asset_type = '')
            OR (
                site.protocol_id = 34
                AND asset_type = 'el'
            )
            OR (
                site.protocol_id = 35
                AND asset_type = 'ext'
                AND toc.lcs_device_type != 'Other'
                AND toc.type != 'Fire Blanket'
            )
        )
        AND test.created_at BETWEEN task_template.created_at AND IFNULL(task_template.deleted_at, CURRENT_TIMESTAMP())
        AND task_template.type IN (1,3)
        LEFT JOIN topaz.api_service_tasks_record AS task_record
        ON task_record.task_id = task_template.id
        AND task_record.test_id = test.id
        WHERE task_record.id IS NULL
        AND test.created_at >= '2024-01-01 00:00:00'
        AND site.id = %(site_id)s
        """,
        {"site_id": site_id}
    )
    print(f"---> {problem_tests} problem tests")
    if problem_tests > 0:
        problem_sites.append({
            "site_id": site_id,
            "problem_tests": problem_tests
        })

Site 4169
---> 0 problem tests
Site 4308
---> 0 problem tests
Site 5344
---> 0 problem tests
Site 5802
---> 0 problem tests
Site 8069
---> 0 problem tests
Site 8295
---> 0 problem tests
Site 8296
---> 0 problem tests
Site 8466
Skipping known problematic site
Site 8467
---> 0 problem tests
Site 8515
---> 0 problem tests
Site 8754
---> 0 problem tests
Site 8757
---> 0 problem tests
Site 8758
---> 0 problem tests
Site 8917
---> 1 problem tests
Site 8928
---> 0 problem tests
Site 9072
---> 0 problem tests
Site 9074
---> 0 problem tests
Site 9085
---> 0 problem tests
Site 9086
---> 0 problem tests
Site 9226
---> 0 problem tests
Site 9561
---> 6 problem tests
Site 9569
---> 0 problem tests
Site 9757
---> 10 problem tests
Site 9943
---> 0 problem tests
Site 9963
---> 0 problem tests
Site 10303
---> 0 problem tests
Site 10700
---> 0 problem tests
Site 16051
---> 289 problem tests
Site 17029
---> 0 problem tests
Site 17114
---> 0 problem tests
Site 17139
---> 0 problem tests
Site 17180
---> 0 p

In [ ]:
18393